# BERTの学習・推論・判定根拠可視化
IMDbのポジネガ判定をBERTでやってみる  
分類タスク用のアダプターモジュールを追加してファインチューニングする  
また，Self-Attentionの重みを可視化し，推論で重要となる単語をハイライト

## IMDbデータを読み込み，DataLoaderを作成
7章と異なる点があるのでここで再実装
- Bert用のWordPieceを用いてサブワードに対応したTokenizerを使用
- 訓練データに含まれている単語ではなく，BERTが持つ全単語を使用
    - BERTEmbeddingモジュールでは全単語を使用する
    - bert-base-uncased-vocab.txt

In [2]:
import os, re, string
from utils.bert import BertTokenizer

data_dir = "../../datasets/ptca_datasets/chapter8"
imdb_dir = os.path.join(data_dir, "aclImdb")
vocab_dir = os.path.join(data_dir, "vocab")
weights_dir = os.path.join(data_dir, "weights")
vocab_save_path=os.path.join(vocab_dir, "bert-base-uncased-vocab.txt")
weights_save_path = os.path.join(weights_dir, "pytorch_model.bin")
config_save_path = os.path.join(weights_dir, "bert_config.json")

In [3]:
# IMDbの前処理(7章と同じ)
def preprocessing_text(text):
    text = re.sub('<br />', '', text)
    
    # カンマ，ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p ==","):
            # ピリオドとカンマの前後にはスペースを入れる
            text = text.replace(p, f" {p} ")
        else:
            text = text.replace(p, " ")
    
    return text

# 違うのはTokenizerがサブワード対応＆BERTのボキャブラリを使用していること
tokenizer_bert = BertTokenizer(vocab_save_path)

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    return tokenizer(text)

データを読み込んだ時の処理をTEXT，LABELとして用意  
max_length=256で，BERTに入力するとき`<PAD>`を入れて512単語にする  
(SEPで2文に分割することはしない)

In [6]:
import torchtext

max_length = 256

TEXT = torchtext.data.Field(
    sequential=True,
    tokenize=tokenizer_with_preprocessing,
    use_vocab=True,
    lower=True,
    include_lengths=True,
    batch_first=True,
    fix_length=max_length,
    init_token="[CLS]",
    eos_token="[SEP]",
    pad_token="[PAD]",
    unk_token="[UNK]"
)

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

IMDbを整形したtsvファイルを読み込み，Datasetにする

In [ ]:
import random

train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path=imdb_dir,
    train="IMDb_train.tsv",
    test="IMDb_test.tsv",
    format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)]
)

train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))